Anime recommendation system by using sklearn nearest neighbours algorithm. The users data is only a small subset of users from MyAnimeList due to resource constraint.

In [1]:
!git clone https://github.com/KenNL42/mal_recommendation

fatal: destination path 'mal_recommendation' already exists and is not an empty directory.


Get my own MyAnimeList Data

In [2]:
import requests
import csv

data = []

username = 'l3roken99' # replace this with your username
user_id = -1

api_path = 'https://api.jikan.moe/v3'
endpoint = '/user/' + username + '/animelist/completed'
url = api_path + endpoint

page = requests.get(url)
anime_list = page.json()['anime']

for anime in anime_list:
  row = []
  row.append(user_id)
  row.append(anime['title'])
  row.append(anime['score'])

  data.append(row)


fields = ['user_id', 'title', 'score']
filename = 'my_own_score.csv'

with open(filename, 'w') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(fields)
  csvwriter.writerows(data)

Transfer users data to pandas

In [3]:
import os
import pandas as pd
import numpy as np

data_path = os.path.join("./mal_recommendation", 'user_scores.csv')

df = pd.read_csv(data_path)

ken_df = pd.read_csv(os.path.join("./",'my_own_score.csv'))

df = pd.concat([df, ken_df], ignore_index=True)

In [4]:
df.head(10)

,user_id,title,score
0,0,5-toubun no Hanayome ∬,0
1,0,Ajin,0
2,0,Ajin 2nd Season,0
3,0,Black Clover,10
4,0,Danshi Koukousei no Nichijou,0
5,0,Dr. Stone: Stone Wars,0
6,0,Hataraku Saibou Black (TV),0
7,0,Horimiya,0
8,0,Hortensia Saga (TV),0
9,0,Jaku-Chara Tomozaki-kun,0


Drop data that scores 0, since it means nothing and can reduce feature

In [5]:
df_train = df[df.score != 0]

print('shape: {}'.format(np.shape(df_train)))
df_train.head()

shape: (76413, 3)


,user_id,title,score
3,0,Black Clover,10
22,1,Akame ga Kill!,7
23,1,B: The Beginning,7
24,1,Black Lagoon,10
25,1,Black Lagoon Omake,10


One hot encoding titles

In [6]:
one_hot = pd.get_dummies(df_train['title'])
print("Dataframe after one-hot encoding: {}".format(np.shape(one_hot)))
one_hot.head()

Dataframe after one-hot encoding: (76413, 5188)


,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie","""Calpis"" Hakkou Monogatari","""Eiji""","""Eiyuu"" Kaitai","""Kiss Dekiru Gyoza"" x Mameshiba Movie","""Parade"" de Satie","""R100"" x Mameshiba Original Manners","""Star""t",** Kouhai,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//The Movie: Sekai no Mukou ni,.hack//Unison,.hack//Versus: The Thanatos Report,009 Re:Cyborg,009-1,009-1: R&B,07-Ghost,0:08,1+2=Paradise,1-gatsu ni wa Christmas,1/100 Shibuya Crossing,1/100 Train Station,100% Renewable Energy,100-man no Inochi no Ue ni Ore wa Tatteiru,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,...,Yu☆Gi☆Oh! VRAINS,Yu☆Gi☆Oh! Zexal,Yu☆Gi☆Oh!: Duel Monsters GX,Yu☆Gi☆Oh!: Hikari no Pyramid,Zankyou no Terror,Zero kara Hajimeru Mahou no Sho,Zero no Tsukaima,Zero no Tsukaima F,Zero no Tsukaima: Futatsuki no Kishi,Zero no Tsukaima: Princesses no Rondo,Zero no Tsukaima: Princesses no Rondo - Yuuwaku no Sunahama,Zetman,Zetsuen no Tempest,Zetsumetsu Kigu Shoujo: Amazing Twins,Zettai Junshu☆Kyousei Kozukuri Kyokashou!!,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zinba,Zipang,Zoku Natsume Yuujinchou,Zoku Owarimonogatari,Zombie-Loan,Zombieland Saga,Zombieland Saga: Revenge,Zton Jingai Animation: A Beautiful Greed Nulu Nulu,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,_Summer,eX-Driver,ef: A Tale of Melodies.,ef: A Tale of Melodies. - Prologue,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,number24,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,ēlDLIVE,◯
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Drop features that has one frequency of 1 in its column, since it is a rare occasion

In [7]:
once_freq_column = one_hot.astype(bool).sum(axis=0).loc[one_hot.astype(bool).sum(axis=0) == 1].index
one_hot.drop(once_freq_column, axis=1, inplace=True)
print("New shape after dropping rare column: {}".format(np.shape(one_hot)))

New shape after dropping rare column: (76413, 3511)


Merge score and title features dataframes together

In [8]:
joined_pd = pd.concat([df_train, one_hot], axis=1)
joined_pd.head()

,user_id,title,score,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie","""Eiyuu"" Kaitai",.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//Unison,.hack//Versus: The Thanatos Report,009 Re:Cyborg,07-Ghost,0:08,1+2=Paradise,100-man no Inochi no Ue ni Ore wa Tatteiru,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,11eyes,11eyes: Momoiro Genmutan,12-sai.,12-sai. 2nd Season,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,18if,2.43: Seiin Koukou Danshi Volley-bu,20-dai no Heya-hen,22/7,...,Yuuki Yuuna wa Yuusha de Aru,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou,Yuuki Yuuna wa Yuusha de Aru: Yuusha no Shou,Yuukoku no Moriarty,Yuukoku no Moriarty 2nd Season,Yuusha ni Narenakatta Ore wa Shibushibu Shuushoku wo Ketsui Shimashita.,Yuusha ni Narenakatta Ore wa Shibushibu Shuushoku wo Ketsui Shimashita. OVA,Yuu☆Yuu☆Hakusho,Yuyushiki,Yu☆Gi☆Oh!,Yu☆Gi☆Oh! 5D's,Yu☆Gi☆Oh! Arc-V,Yu☆Gi☆Oh! Duel Monsters,Yu☆Gi☆Oh! Movie: Chou Yuugou! Toki wo Koeta Kizuna,Yu☆Gi☆Oh! The Dark Side of Dimensions,Yu☆Gi☆Oh! Zexal,Yu☆Gi☆Oh!: Duel Monsters GX,Yu☆Gi☆Oh!: Hikari no Pyramid,Zankyou no Terror,Zero kara Hajimeru Mahou no Sho,Zero no Tsukaima,Zero no Tsukaima F,Zero no Tsukaima: Futatsuki no Kishi,Zero no Tsukaima: Princesses no Rondo,Zero no Tsukaima: Princesses no Rondo - Yuuwaku no Sunahama,Zetsuen no Tempest,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zoku Natsume Yuujinchou,Zoku Owarimonogatari,Zombieland Saga,Zombieland Saga: Revenge,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,number24,s.CRY.ed,xxxHOLiC,ēlDLIVE,◯
3,0,Black Clover,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,1,Akame ga Kill!,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,1,B: The Beginning,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,1,Black Lagoon,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25,1,Black Lagoon Omake,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


replace 1's with scores instead

In [9]:
# replace columns of 1's with score for that one

after_mult = pd.DataFrame(joined_pd.loc[:, joined_pd.columns[3:]].values * [[a] for a in joined_pd.iloc[:,2].values])

after_mult.columns = joined_pd.columns[3:]
after_mult.index = joined_pd.index

joined_pd[joined_pd.columns[3:]] = after_mult[after_mult.columns]

In [10]:
'''
# takes 2:30 min
test = joined_pd.copy()
for i in test.index:
  # title, score = test.loc[i]['title'], test.loc[i]['score']

  test.at[i, test.loc[i]['title']] = test.loc[i]['score']
  '''

"\n# takes 2:30 min\ntest = joined_pd.copy()\nfor i in test.index:\n  # title, score = test.loc[i]['title'], test.loc[i]['score']\n\n  test.at[i, test.loc[i]['title']] = test.loc[i]['score']\n  "

get data that has titles and score

In [11]:
data = joined_pd.drop(columns=['title', 'score'])
data.head()

,user_id,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie","""Eiyuu"" Kaitai",.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//Unison,.hack//Versus: The Thanatos Report,009 Re:Cyborg,07-Ghost,0:08,1+2=Paradise,100-man no Inochi no Ue ni Ore wa Tatteiru,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,11eyes,11eyes: Momoiro Genmutan,12-sai.,12-sai. 2nd Season,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,18if,2.43: Seiin Koukou Danshi Volley-bu,20-dai no Heya-hen,22/7,22/7: 8＋3=?,25-sai no Joshikousei,...,Yuuki Yuuna wa Yuusha de Aru,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou,Yuuki Yuuna wa Yuusha de Aru: Yuusha no Shou,Yuukoku no Moriarty,Yuukoku no Moriarty 2nd Season,Yuusha ni Narenakatta Ore wa Shibushibu Shuushoku wo Ketsui Shimashita.,Yuusha ni Narenakatta Ore wa Shibushibu Shuushoku wo Ketsui Shimashita. OVA,Yuu☆Yuu☆Hakusho,Yuyushiki,Yu☆Gi☆Oh!,Yu☆Gi☆Oh! 5D's,Yu☆Gi☆Oh! Arc-V,Yu☆Gi☆Oh! Duel Monsters,Yu☆Gi☆Oh! Movie: Chou Yuugou! Toki wo Koeta Kizuna,Yu☆Gi☆Oh! The Dark Side of Dimensions,Yu☆Gi☆Oh! Zexal,Yu☆Gi☆Oh!: Duel Monsters GX,Yu☆Gi☆Oh!: Hikari no Pyramid,Zankyou no Terror,Zero kara Hajimeru Mahou no Sho,Zero no Tsukaima,Zero no Tsukaima F,Zero no Tsukaima: Futatsuki no Kishi,Zero no Tsukaima: Princesses no Rondo,Zero no Tsukaima: Princesses no Rondo - Yuuwaku no Sunahama,Zetsuen no Tempest,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zoku Natsume Yuujinchou,Zoku Owarimonogatari,Zombieland Saga,Zombieland Saga: Revenge,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,number24,s.CRY.ed,xxxHOLiC,ēlDLIVE,◯
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
data = data.groupby("user_id").sum()
print("After grouping each users into one row, the shape becomes: {}".format(np.shape(data)))
data.head()

After grouping each users into one row, the shape becomes: (620, 3511)


,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie","""Eiyuu"" Kaitai",.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//Unison,.hack//Versus: The Thanatos Report,009 Re:Cyborg,07-Ghost,0:08,1+2=Paradise,100-man no Inochi no Ue ni Ore wa Tatteiru,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,11eyes,11eyes: Momoiro Genmutan,12-sai.,12-sai. 2nd Season,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,18if,2.43: Seiin Koukou Danshi Volley-bu,20-dai no Heya-hen,22/7,22/7: 8＋3=?,25-sai no Joshikousei,3-gatsu no Lion,...,Yuuki Yuuna wa Yuusha de Aru,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou,Yuuki Yuuna wa Yuusha de Aru: Yuusha no Shou,Yuukoku no Moriarty,Yuukoku no Moriarty 2nd Season,Yuusha ni Narenakatta Ore wa Shibushibu Shuushoku wo Ketsui Shimashita.,Yuusha ni Narenakatta Ore wa Shibushibu Shuushoku wo Ketsui Shimashita. OVA,Yuu☆Yuu☆Hakusho,Yuyushiki,Yu☆Gi☆Oh!,Yu☆Gi☆Oh! 5D's,Yu☆Gi☆Oh! Arc-V,Yu☆Gi☆Oh! Duel Monsters,Yu☆Gi☆Oh! Movie: Chou Yuugou! Toki wo Koeta Kizuna,Yu☆Gi☆Oh! The Dark Side of Dimensions,Yu☆Gi☆Oh! Zexal,Yu☆Gi☆Oh!: Duel Monsters GX,Yu☆Gi☆Oh!: Hikari no Pyramid,Zankyou no Terror,Zero kara Hajimeru Mahou no Sho,Zero no Tsukaima,Zero no Tsukaima F,Zero no Tsukaima: Futatsuki no Kishi,Zero no Tsukaima: Princesses no Rondo,Zero no Tsukaima: Princesses no Rondo - Yuuwaku no Sunahama,Zetsuen no Tempest,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zoku Natsume Yuujinchou,Zoku Owarimonogatari,Zombieland Saga,Zombieland Saga: Revenge,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,number24,s.CRY.ed,xxxHOLiC,ēlDLIVE,◯
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##Create model with Nearestneighbors

In [13]:
from sklearn.neighbors import NearestNeighbors

train_data = data.iloc[1:]
test_data = data.iloc[0]

# neighbor_num = 20

model_knn = NearestNeighbors()

model_knn.fit(train_data)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [14]:
#display data to double check
print("My MAL: ")
print(test_data.loc[test_data != 0])

My MAL: 
Akira                                                        7
Angel Beats!                                                 7
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.    10
Ansatsu Kyoushitsu                                           8
Araburu Kisetsu no Otome-domo yo.                            9
                                                            ..
Yagate Kimi ni Naru                                          8
Yesterday wo Utatte                                          9
Yojouhan Shinwa Taikei                                       9
Yuru Camp△                                                   8
Zombieland Saga                                              9
Name: -1, Length: 113, dtype: int64


Get nearest neighbors and users id

In [15]:
distances, indices = model_knn.kneighbors(np.array(test_data).reshape(1, -1), n_neighbors=50)

In [16]:
distances, indices

(array([[83.51047838, 83.8987485 , 84.29709366, 84.47484833, 85.29361055,
         85.29361055, 85.37564055, 85.41077215, 85.52777327, 85.76129663,
         85.86035173, 85.87782019, 85.87782019, 85.88364221, 85.94765849,
         85.97092532, 86.04649906, 86.04649906, 86.04649906, 86.08716513,
         86.14522622, 86.22064718, 86.23224455, 86.23224455, 86.25543461,
         86.28441342, 86.32496742, 86.34234187, 86.34234187, 86.34234187,
         86.34234187, 86.34234187, 86.34234187, 86.34813258, 86.44073114,
         86.44073114, 86.52167359, 86.62563131, 86.66602564, 86.70063437,
         86.71216754, 86.7986175 , 86.81589716, 86.82741503, 86.85044617,
         86.89073598, 86.91950299, 86.91950299, 86.93675862, 87.01723967]]),
 array([[580, 142,  41, 492, 119, 489, 166, 101,  90, 398, 599, 417,  83,
         385, 536, 435, 600, 338, 290, 368, 134, 537, 327, 412, 253, 596,
         401,   0, 460, 326, 214,  40, 475, 345, 188, 506, 233, 549, 219,
         289, 471, 464, 260, 453, 2

Process data by creating a dict of {titles: [score_list]}

In [18]:
from collections import defaultdict
import itertools

filtered_test_data = test_data.loc[test_data != 0]
all_neighbors_dict = defaultdict(list)

for i in indices[0]:
  near_user_score_df = train_data.iloc[i].loc[train_data.iloc[i] != 0]
  near_user_score_dict = near_user_score_df[~near_user_score_df.index.isin(filtered_test_data.index)].to_dict()

  for key, val in itertools.chain(near_user_score_dict.items()):
    all_neighbors_dict[key].append(val)

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(all_neighbors_dict)

defaultdict(<class 'list'>, {'Boku dake ga Inai Machi': [7, 8], 'Death Note': [9, 8, 8, 7, 9, 10, 10, 8], 'Devilman: Crybaby': [7, 10], 'FLCL': [10, 8, 10], 'Ijiranaide, Nagatoro-san': [7], 'K-On!: Live House!': [7], 'Kaguya-sama wa Kokurasetai?: Tensai-tachi no Renai Zunousen': [8, 10, 9, 8], 'Kiseijuu: Sei no Kakuritsu': [9, 5], 'Koyomimonogatari': [7, 8, 7], 'Neon Genesis Evangelion: The End of Evangelion': [10, 9, 9, 10], 'Owarimonogatari': [8, 9, 10], 'Owarimonogatari 2nd Season': [10, 6, 10], 'Shingeki no Kyojin': [8, 7, 8, 8], 'Shingeki no Kyojin Season 2': [9, 3, 9, 8], 'Shingeki no Kyojin Season 3': [7, 4, 9, 8], 'Shingeki no Kyojin Season 3 Part 2': [9, 6, 9, 9, 10, 9, 10], 'Shingeki no Kyojin: The Final Season': [8, 8, 10, 7, 9], 'Zoku Owarimonogatari': [9, 8, 7], 'Mirai Nikki (TV)': [9, 9], 'Another': [2], 'Aria the Animation': [6], 'Barakamon': [9, 10], 'Black Lagoon': [4], 'Black Lagoon: The Second Barrage': [4], 'Carnival Phantasm': [9], 'Clannad: After Story': [10, 8], 

Average each title scores

In [20]:
average_neighbors_score = defaultdict(list) # key: anime name, value: list([average, length])
for key, val in all_neighbors_dict.items():
  average_neighbors_score[key] = [np.mean(val), len(val)]

Show top 10 highest rating and top 10 most mentioned

In [21]:
import operator

sorted_list = sorted(average_neighbors_score.items(), key=lambda x: x[1][0], reverse=True)

print('Top 10 nearest neighbors anime and highest average score: ')
for i in range(10):
  print("{} \t score: {}\t with frequency: {}".format(sorted_list[i][0], sorted_list[i][1][0], sorted_list[i][1][1]))

sorted_freq_list = sorted(average_neighbors_score.items(), key=lambda x: x[1][1], reverse=True)
print('\nTop 10 nearest neighbors anime and highest frequency: ')
for i in range(10):
  print("{} \t score: {}\t with frequency: {}".format(sorted_freq_list[i][0], sorted_freq_list[i][1][0], sorted_freq_list[i][1][1]))

Top 10 nearest neighbors anime and highest average score: 
Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita 	 score: 10.0	 with frequency: 1
Ginga Eiyuu Densetsu 	 score: 10.0	 with frequency: 1
Monster 	 score: 10.0	 with frequency: 1
Black Clover 	 score: 10.0	 with frequency: 2
Cardcaptor Sakura: Clear Card-hen 	 score: 10.0	 with frequency: 1
Kill la Kill 	 score: 10.0	 with frequency: 1
Tonari no Kaibutsu-kun 	 score: 10.0	 with frequency: 1
Nisekoi 	 score: 10.0	 with frequency: 1
Ishuzoku Reviewers 	 score: 10.0	 with frequency: 1
Cowboy Bebop: Tengoku no Tobira 	 score: 10.0	 with frequency: 1

Top 10 nearest neighbors anime and highest frequency: 
Death Note 	 score: 8.625	 with frequency: 8
Shingeki no Kyojin Season 3 Part 2 	 score: 8.857142857142858	 with frequency: 7
Mob Psycho 100 II 	 score: 9.166666666666666	 with frequency: 6
Jujutsu Kaisen (TV) 	 score: 7.833333333333333	 with frequency: 6
Shingeki no Kyojin: The Final Season 	 score: 8.4	 with frequ